###### The original MATLAB code was developed by Prof. Hyunseok Oh (Gwangju Institute of Science and Technology, South Korea, hsoh@gist.ac.kr).

# Feature Extraction

In [1]:
##### Add subfunction ----------------------------------------------------------------------------
source('subfunction/positiveFFT.R')
source('subfunction/Find_Freqmagnitude.R')
source('subfunction/Make_Featureset.R')

In [3]:
##### Data load ----------------------------------------------------------------------------------
unbalance <- R.matlab::readMat('../data_files/unbalance.mat')$unbalance
misalign <- R.matlab::readMat('../data_files/misalign.mat')$misalign
outer <- R.matlab::readMat('../data_files/outerrace_fault.mat')$outer
inner <- R.matlab::readMat('../data_files/innerrace_fault.mat')$inner

In [4]:
Time <- unbalance[, 1]
N = length(Time) / 10  # divide to 10 split(1 sec * 10)
unbalance02 <- {}

Fs <- 5120
k <- 0:(N - 1)
P <- N / Fs  # period = 1 / frequency
freq <- k[1:ceiling(N / 2)] / P

Freq_unbalance02 <- {}
for (i in 1:10) {
  temp_data02 <- unbalance[((i - 1) * N + 1):(i * N), 2]
  unbalance02 <- cbind(unbalance02, temp_data02)
  Freq_unbalance02 <- cbind(Freq_unbalance02, positiveFFT(temp_data02, Fs))
}

In [5]:
# Time-domain health features
TF_Max <- apply(unbalance02, 2, max)
TF_Mean <- colMeans(unbalance02)
TF_RMS <- apply(unbalance02, 2, function(x) sqrt(mean(x ^ 2)))
TF_Skewness <- moments::skewness(unbalance02)
TF_Kurtosis <- moments::kurtosis(unbalance02)
TF_CrestFactor <- TF_Max / TF_RMS
TF_ShapeFactor <- TF_Max / TF_Mean
TF_ImpulseFactor <- TF_RMS / TF_Mean

Time_Featureset <- cbind(TF_Max, TF_Mean, TF_RMS, TF_Skewness, TF_Kurtosis,
                         TF_CrestFactor, TF_ShapeFactor, TF_ImpulseFactor)

In [6]:
Time_Featureset

,TF_Max,TF_Mean,TF_RMS,TF_Skewness,TF_Kurtosis,TF_CrestFactor,TF_ShapeFactor,TF_ImpulseFactor
temp_data02,0.6872157,0.002612340,0.2515242,-0.1608559,2.602627,2.732205,263.06522,96.28313
temp_data02,0.5983858,0.006449014,0.2374259,-0.2184838,2.420470,2.520305,92.78718,36.81585
temp_data02,0.6043004,0.007923288,0.2373216,-0.2149297,2.468927,2.546336,76.26890,29.95241
temp_data02,0.6743609,0.008115311,0.2533792,-0.1803572,2.604153,2.661470,83.09736,31.22236
temp_data02,0.5986238,0.007099021,0.2440969,-0.1917067,2.426332,2.452402,84.32485,34.38459
temp_data02,0.6518498,0.004353759,0.2439174,-0.2294876,2.468173,2.672420,149.72113,56.02456
temp_data02,0.6260913,0.002397111,0.2465280,-0.1811897,2.583367,2.539636,261.18583,102.84380
temp_data02,0.5613108,0.002325913,0.2405842,-0.2208770,2.419844,2.333116,241.32918,103.43643
temp_data02,0.6051062,0.002986301,0.2459793,-0.1794789,2.532991,2.459988,202.62735,82.36924
temp_data02,0.6615366,0.002448249,0.2437932,-0.1766365,2.567290,2.713515,270.20811,99.57862


In [10]:
# Freq.-domain health features
RPM <- 3600
Freq_1x <- RPM / 60
theta <- 0  # Angular (degrees)
Bd <- 6.747  # Ball diameter (mm)
Pd <- 28.5  # Pitch diameter (mm)
N <- 8  # Number of bearing ball
FTF <- (Freq_1x / 2) * (1 - Bd * cos(theta) / Pd)  # Fundamental train Frequency or Cage Frequency
BPFI <- (Freq_1x * N / 2) * (1 + Bd * cos(theta) / Pd)  # Ball Pass Frequency of Inner Race
BPFO <- (Freq_1x * N / 2) * (1 - Bd * cos(theta) / Pd)  # Ball Pass Frequency of Outer Race
BSF <- (Freq_1x * Pd / (2 * Bd)) * (1 - (Bd * cos(theta) / Pd) ^ 2)  # Ball Spin Frequency

In [11]:
Magnitude_under0p49x <- Find_Freqmagnitude(0:(Freq_1x * 0.49), freq, Freq_unbalance02)
Magnitude_0p51to0p99x <- Find_Freqmagnitude((Freq_1x * 0.51):(Freq_1x * 0.99), freq, Freq_unbalance02)
Magnitude_3to5x <- Find_Freqmagnitude((Freq_1x * 3):(Freq_1x * 5), freq, Freq_unbalance02)
Magnitude_1to10x <- Find_Freqmagnitude((Freq_1x):(Freq_1x * 10), freq, Freq_unbalance02)
Magnitude_FTF <- Find_Freqmagnitude(FTF, freq, Freq_unbalance02)
Magnitude_BPFI <- Find_Freqmagnitude(BPFI, freq, Freq_unbalance02)
Magnitude_BPFO <- Find_Freqmagnitude(BPFO, freq, Freq_unbalance02)
Magnitude_BSF <- Find_Freqmagnitude(BSF, freq, Freq_unbalance02)
Magnitude_0p5x <- Find_Freqmagnitude(Freq_1x * 0.5, freq, Freq_unbalance02)
Magnitude_1X <- Find_Freqmagnitude(Freq_1x, freq, Freq_unbalance02)
Magnitude_2X <- Find_Freqmagnitude(Freq_1x * 2, freq, Freq_unbalance02)
Magnitude_3X <- Find_Freqmagnitude(Freq_1x * 3, freq, Freq_unbalance02)
Magnitude_5X <- Find_Freqmagnitude(Freq_1x * 5, freq, Freq_unbalance02)
Magnitude_7X <- Find_Freqmagnitude(Freq_1x * 7, freq, Freq_unbalance02)
Magnitude_9X <- Find_Freqmagnitude(Freq_1x * 9, freq, Freq_unbalance02)

In [15]:
FF_a <- Magnitude_1X
FF_b <- Magnitude_under0p49x / Magnitude_1X
FF_c <- Magnitude_0p5x / Magnitude_1X
FF_d <- Magnitude_0p51to0p99x / Magnitude_1X
FF_e <- Magnitude_2X / Magnitude_1X
FF_f <- Magnitude_3to5x / Magnitude_1X
FF_g <- apply(rbind(Magnitude_3X, Magnitude_5X, Magnitude_7X, Magnitude_9X), 2, mean) / Magnitude_1X
FF_h <- Magnitude_1to10x / Magnitude_1X
FF_FTF <- Magnitude_FTF
FF_BPFI <- Magnitude_BPFI
FF_BPFO <- Magnitude_BPFO
FF_BSF <- Magnitude_BSF

Freq_Featureset <- cbind(FF_a, FF_b, FF_c, FF_d, FF_e, FF_f, FF_g, FF_h, FF_FTF, FF_BPFI, FF_BPFO, FF_BSF)
Freq_Featureset

FF_a,FF_b,FF_c,FF_d,FF_e,FF_f,FF_g,FF_h,FF_FTF,FF_BPFI,FF_BPFO,FF_BSF
0.01277658,0.11592280,0.06559776,0.07159565,0.7507004,0.8163060,6.425234,0.4191231,0.0020721861,0.013422324,0.2195551,0.009591382
0.01181285,0.14281229,0.02650091,0.06741297,0.6221931,0.8309285,6.650403,0.4071264,0.0005439719,0.011624367,0.2168728,0.007349873
0.01234337,0.11333042,0.10857923,0.07146418,0.7722869,0.7558330,7.353509,0.3647518,0.0007013515,0.007229164,0.2513442,0.009532621
0.01254206,0.15690694,0.13540159,0.11413795,0.7207096,0.7672491,7.648861,0.3950344,0.0011519443,0.007412825,0.2740114,0.009039179
0.01153667,0.12844015,0.03406002,0.05990051,0.6666423,0.7752394,8.278668,0.4104048,0.0012156833,0.006374144,0.2824273,0.007690832
0.01312783,0.13579590,0.12822139,0.13025176,0.6038864,0.6611322,7.179871,0.3478556,0.0003779983,0.007195051,0.2878441,0.007927715
0.01246669,0.09664900,0.03635899,0.07133786,0.6217167,0.7292640,7.606843,0.3739359,0.0011768713,0.009122198,0.2679756,0.007750752
0.01138818,0.13320349,0.09471857,0.09254904,0.6275001,0.7094455,8.190392,0.3887325,0.0007248655,0.007199295,0.2834573,0.007146086
0.01300788,0.09371948,0.06258517,0.06702540,0.5750127,0.7264565,7.232555,0.3632571,0.0005733854,0.007528219,0.2714480,0.007479694
0.01182133,0.13039929,0.05599733,0.08759599,0.4438703,0.8063149,7.693607,0.4150746,0.0016022912,0.010714548,0.2491166,0.005247137


In [23]:
Featureset <- cbind(Time_Featureset, Freq_Featureset)

In [18]:
# Unbalance
# Unbalance02_featureset <- Make_Featureset(unbalance[, 2])
Unbalance03_featureset <- Make_Featureset(unbalance[, 3])
Unbalance04_featureset <- Make_Featureset(unbalance[, 4])

# Misalignment
# Misalign02_featureset <- Make_Featureset(misalign[, 2])
Misalign03_featureset <- Make_Featureset(misalign[, 3])

# Innerrace_fault
# Inner02_featureset <- Make_Featureset(inner[, 2])
Inner03_featureset <- Make_Featureset(inner[, 3])
Inner04_featureset <- Make_Featureset(inner[, 4])

# Outerrace_fault
# Outer02_featureset <- Make_Featureset(outer[, 2])
Outer03_featureset <- Make_Featureset(outer[, 3])
Outer04_featureset <- Make_Featureset(outer[, 4])

In [19]:
Featureset_Total <- rbind(Unbalance03_featureset, Unbalance04_featureset,
                          Misalign03_featureset, 
                          Inner03_featureset, Inner04_featureset,
                          Outer03_featureset, Outer04_featureset)
rownames(Featureset_Total) <- NULL

In [21]:
##### Data Save ----------------------------------------------------------------------------------
save(Featureset_Total, file = 'saved_data/Featureset.rds')